In [1]:
import pandas as pd
import os
import logging

# Read the CSV dataset
file_path = './datasets/movies.csv'
df = pd.read_csv(file_path) # Add .head(100) if you want to limit the number of rows

# Perform the data cleaning
def clean_dataset(text, sep_token=' \n '):
    if pd.isna(text):
        return ''
    else:
        text = str(text)
        text = text.strip()
    return text

df['title'] = df['title'].apply(clean_dataset)
df['summary'] = df['summary'].apply(clean_dataset)

df = df.fillna('')

print(str(len(df)) + ' rows')
df.head()

40179 rows


,imdb_id,title,summary,date,genres,runtime,rating,votes,budget,revenue,language,adult,production,poster_link
0,114709,Toy Story (1995),"Led by Woody, Andy's toys live happily in his ...",10/30/95,"Animation, Adventure, Comedy",81.0,8.3,5415.0,30000000,373554033.0,en,False,Pixar Animation Studios,https://images-na.ssl-images-amazon.com/images...
1,113497,Jumanji (1995),When siblings Judy and Peter discover an encha...,12/15/95,"Action, Adventure, Family",104.0,6.9,2413.0,65000000,262797249.0,en,False,"TriStar Pictures, Teitler Film, Interscope Com...",https://images-na.ssl-images-amazon.com/images...
2,113228,Grumpier Old Men (1995),A family wedding reignites the ancient feud be...,12/22/95,"Comedy, Romance",101.0,6.6,92.0,0,0.0,en,False,"Warner Bros., Lancaster Gate",https://images-na.ssl-images-amazon.com/images...
3,114885,Waiting to Exhale (1995),"Cheated on, mistreated and stepped on, the wom...",12/22/95,"Comedy, Drama, Romance",127.0,5.7,34.0,16000000,81452156.0,en,False,Twentieth Century Fox Film Corporation,https://images-na.ssl-images-amazon.com/images...
4,113041,Father of the Bride Part II (1995),Just when George Banks has recovered from his ...,2/10/95,"Comedy, Family, Romance",106.0,5.9,173.0,0,76578911.0,en,False,"Sandollar Productions, Touchstone Pictures",https://images-na.ssl-images-amazon.com/images...


In [5]:
# Create a new column named 'metadata' with all the data combined
def generate_metadata(row):
    metadata = (
        f"The title of this film is '{row['title']}'. "
        f"The brief summary of this film is '{row['summary']}'. "
        f"The release date of this film is '{row['date']}'. "
        f"The genre(s) of this film is/are '{row['genres']}'. "
        f"The runtime or length of this film is {row['runtime']} minute(s). "
        f"This film has a rating of {row['rating']} out of 10. "
        f"The budget of this film is {row['budget']} and it made a revenue of {row['revenue']}. "
        f"The language of this film is '{row['language']}'. "
        f"The production company of this film is '{row['production']}'."
    )
    return metadata

df['metadata'] = df.apply(generate_metadata, axis=1)

pd.set_option('display.max_colwidth', None)
print("A sample value from column 'metadata':")
df['metadata'].head(1)

A sample value from column 'metadata':


0    The title of this film is 'Toy Story (1995)'. The brief summary of this film is 'Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.'. The release date of this film is '10/30/95'. The genre(s) of this film is/are 'Animation, Adventure, Comedy'. The runtime or length of this film is 81.0 minute(s). This film has a rating of 8.3 out of 10. The budget of this film is 30000000 and it made a revenue of 373554033.0. The language of this film is 'en'. The production company of this film is 'Pixar Animation Studios'.
Name: metadata, dtype: object

In [6]:
from openai import OpenAI
import openai
from tqdm import tqdm
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY'),
)

# Generate the vector embeddings for selected columns
def get_embedding(text, model='text-embedding-3-small'):
    try:
        text = text.replace('\n', ' ')
        return client.embeddings.create(input = [text], model=model).data[0].embedding
    except Exception as e:
        # logging.error(f'Error generating embeddings: {e}. Found issue in the following text: {text}.')
        text = 'No data available'
        return client.embeddings.create(input = [text], model=model).data[0].embedding

tqdm.pandas()
df['metadata_vector'] = df['metadata'].apply(lambda x: get_embedding(x, model='text-embedding-3-small'))

print('Conversion to vector embeddings has been completed.')

df.head()

Conversion to vector embeddings has been completed.


,imdb_id,title,summary,date,genres,runtime,rating,votes,budget,revenue,language,adult,production,poster_link,metadata,metadata_vector
0,114709,Toy Story (1995),"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",10/30/95,"Animation, Adventure, Comedy",81.0,8.3,5415.0,30000000,373554033.0,en,False,Pixar Animation Studios,"https://images-na.ssl-images-amazon.com/images/M/MV5BMDU2ZWJlMjktMTRhMy00ZTA5LWEzNDgtYmNmZTEwZTViZWJkXkEyXkFqcGdeQXVyNDQ2OTk4MzI@._V1_UX182_CR0,0,182,268_AL_.jpg","The title of this film is 'Toy Story (1995)'. The brief summary of this film is 'Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.'. The release date of this film is '10/30/95'. The genre(s) of this film is/are 'Animation, Adventure, Comedy'. The runtime or length of this film is 81.0 minute(s). This film has a rating of 8.3 out of 10. The budget of this film is 30000000 and it made a revenue of 373554033.0. The language of this film is 'en'. The production company of this film is 'Pixar Animation Studios'.","[-0.010617855004966259, 0.027610644698143005, -0.05589677765965462, -0.05243489146232605, -0.031494710594415665, -0.029721548780798912, 0.016813362017273903, 0.056952230632305145, -0.016496727243065834, -0.01569458283483982, -0.015135192312300205, 0.040339402854442596, -0.01767883263528347, -0.01882927678525448, -0.04551112279295921, 0.0074145556427538395, -0.03341563418507576, -0.02739955298602581, -0.01981084793806076, -0.04850860685110092, 0.034555524587631226, 0.040107205510139465, -0.021721217781305313, -0.005303650163114071, 0.03744746372103691, -0.00918771605938673, -0.018153786659240723, 0.017087779939174652, -0.008385571651160717, 0.03411223366856575, -0.022185616195201874, -0.014406929723918438, 0.017003344371914864, -0.004704680759459734, 0.03476661443710327, -0.014132511802017689, -0.008865803480148315, -0.060287460684776306, 0.018544305115938187, -0.0486774817109108, -0.06075185909867287, 0.0016781698213890195, 0.006929047405719757, 0.00290513364598155, -0.004395960830152035, 0.02355770580470562, -0.031220292672514915, 0.016391180455684662, -0.0007289220811799169, 0.0005587302730418742, -0.0229033250361681, 0.029426023364067078, 0.01855485886335373, 0.019916392862796783, -0.010517586953938007, -0.017140552401542664, -0.009710165672004223, -0.021087946370244026, -0.006522697862237692, -0.022945541888475418, -0.007235128432512283, 0.05294150859117508, 0.013150941580533981, 0.03833404555916786, 0.01662338152527809, -0.02387434057891369, -0.050281770527362823, -0.027019590139389038, 0.0012038757558912039, -0.020634101703763008, 0.02615411952137947, 0.07320620119571686, -0.03799629956483841, 0.03721526265144348, 0.01445970218628645, 0.040888238698244095, 0.0003176253230776638, 0.024190977215766907, 0.055474597960710526, -0.012559887953102589, -0.03073478303849697, 0.01975807547569275, -0.013488685712218285, -0.021826762706041336, -0.054588016122579575, 0.01855485886335373, 0.035885393619537354, -0.02725178934633732, -0.008200868032872677, -0.003997527062892914, 0.013488685712218285, -0.007878954522311687, -0.06459371000528336, 0.006068853195756674, 0.019240902736783028, -0.04264029115438461, 0.022227834910154343, 0.0068498882465064526, 0.0601608082652092, -0.005250877235084772, ...]"
1,113497,Jumanji (1995),"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope

In [7]:
# Save embedding model as CSV (optional)
df.to_csv('./datasets/movies_embedding.csv', index=False)

In [8]:
from qdrant_client import models, QdrantClient
from qdrant_client.http import models as rest
from qdrant_client.http.models import Record
from sentence_transformers import SentenceTransformer

# Initialize the Qdrant client
qdrant_client = QdrantClient(':memory:')

# Set the collection name and size
collection_name = 'movies'
vector_size = len(df['metadata_vector'][0])

# Create a collection
qdrant_client.recreate_collection(
    collection_name=collection_name,
    vectors_config={
        'metadata': rest.VectorParams(
            distance=rest.Distance.COSINE,
            size=vector_size,
        ),
    }
)

# Add vectors to the collection
qdrant_client.upsert(
    collection_name=collection_name,
    points=[
        rest.PointStruct(
            id=k,
            vector={
                'metadata': v['metadata_vector'],
            },
            payload=v.to_dict(),
        )
        for k, v in df.iterrows()
    ],
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [9]:
print(qdrant_client.get_collections())
qdrant_client.count(collection_name=collection_name)

collections=[CollectionDescription(name='movies')]


CountResult(count=40179)

In [10]:
# Generate a query embedding and search in Qdrant
def query_qdrant(query, collection_name, vector_name, top_k=5):
    # Creates embedding vector from user query
    completion = openai.embeddings.create(
        input=query,
        model='text-embedding-3-small'  # Be sure to use the same embedding model as the vectors in the collection
    )

    embedded_query = completion.data[0].embedding

    query_results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=(
            vector_name, embedded_query
        ),
        limit=top_k,
    )
    
    return query_results

In [16]:
# Search for similar vectors and store a result
query_results = query_qdrant('scary movies with monsters released after the year 2000', collection_name=collection_name, vector_name='metadata')

for i, vector in enumerate(query_results):
    print(f"{i + 1}. {vector.payload['title']} {vector.payload['poster_link']} (Score: {round(vector.score, 3)})")

1. Monster (2008) https://images-na.ssl-images-amazon.com/images/M/MV5BMTQ2NjIxNTg5N15BMl5BanBnXkFtZTcwNzE0NzAzOA@@._V1_UY268_CR1,0,182,268_AL_.jpg (Score: 0.596)
2. Monster House (2006) https://images-na.ssl-images-amazon.com/images/M/MV5BMTIzNjE1NDg1N15BMl5BanBnXkFtZTcwOTg2NTMzMQ@@._V1_UX182_CR0,0,182,268_AL_.jpg (Score: 0.571)
3. Scary Movie (2000) https://images-na.ssl-images-amazon.com/images/M/MV5BMGEzZjdjMGQtZmYzZC00N2I4LThiY2QtNWY5ZmQ3M2ExZmM4XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg (Score: 0.562)
4. Boogeyman (2005) https://images-na.ssl-images-amazon.com/images/M/MV5BZWE1MDkyNzQtMTNjMC00OTU4LWE5NTctM2M0N2Y1OTE5NDAyXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg (Score: 0.544)
5. How to Make a Monster (2001) https://images-na.ssl-images-amazon.com/images/M/MV5BMTIyODQyMDgzN15BMl5BanBnXkFtZTcwNDI5NTkyMQ@@._V1_UY268_CR2,0,182,268_AL_.jpg (Score: 0.542)


In [17]:
# Format the response as JSON
import json
from datetime import datetime
import locale

# Assume year in 2000's if between 00 to 24, otherwise, in 1900's
def convert_date_format(date_str):
    month, day, year = map(int, date_str.split('/'))
    
    if year >= 0 and year <= 24:
        year += 2000
    else:
        year += 1900

    date_obj = datetime(year, month, day)
    formatted_date = date_obj.strftime('%m/%d/%Y')
    
    return formatted_date

def string_to_array(str):
    arr = str.split(',')
    arr = [arr.strip() for arr in arr]

    return arr

def format_time(minutes_float):
    minutes_int = int(minutes_float)

    hours = minutes_int // 60
    minutes = minutes_int % 60
    
    if hours > 0:
        time_string = f"{hours}h {minutes}m"
    else:
        time_string = f"{minutes}m"
    
    return time_string

def format_as_dollars(number):
    locale.setlocale(locale.LC_ALL, '')

    number = int(number)
    formatted_number = locale.currency(number, grouping=True)
    formatted_number = formatted_number.replace(locale.localeconv()['currency_symbol'], "$")

    return formatted_number

def get_country_name(abbreviation):
    country_names = {'en': 'English', 'fr': 'French', 'zh': 'Chinese', 'it': 'Italian', 'fa': 'Persian', 'nl': 'Dutch', 'de': 'German', 'cn': 'Chinese', 'ar': 'Arabic', 'es': 'Spanish', 'ru': 'Russian', 'sv': 'Swedish', 'ja': 'Japanese', 'ko': 'Korean', 'sr': 'Serbian', 'bn': 'Bengali', 'he': 'Hebrew', 'pt': 'Portuguese', 'wo': 'Wolof', 'ro': 'Romanian', 'hu': 'Hungarian', 'cy': 'Welsh', 'vi': 'Vietnamese'}
    return country_names.get(abbreviation, '')

# Function to search for similar vectors
def search_movies_in_qdrant(query):
    query_results = query_qdrant(query, collection_name, 'metadata')

    results = []
    
    for i, vector in enumerate(query_results):
        tmp = {
            "rank": i,
            "title": vector.payload["title"],
            "summary": vector.payload["summary"],
            "date": convert_date_format(vector.payload["date"]),
            "genres": string_to_array(vector.payload["genres"]),
            "runtime": format_time(vector.payload["runtime"]),
            "rating": vector.payload["rating"],
            "votes": int(vector.payload["votes"]),
            "budget": format_as_dollars(vector.payload["budget"]),
            "revenue": format_as_dollars(vector.payload["revenue"]),
            "language": get_country_name(vector.payload["language"]),
            "adult": vector.payload["adult"],
            "production": string_to_array(vector.payload["production"]),
            "poster_link": vector.payload["poster_link"]
        }
        results.append(tmp)

    return results

In [18]:
query = 'scary movies about monsters after year 2003'
response = (search_movies_in_qdrant(query))

json_string = json.dumps(response, indent=2)
print(json_string)

[
  {
    "rank": 0,
    "title": "Monster (2008)",
    "summary": "Two women, aspiring documentary filmmakers, find themselves trapped in a monster-plagued Toyko in 2003.",
    "date": "01/18/2008",
    "genres": [
      "Action",
      "Horror",
      "Thriller"
    ],
    "runtime": "1h 26m",
    "rating": 2.1,
    "votes": 7,
    "budget": "$0.00",
    "revenue": "$0.00",
    "language": "English",
    "adult": false,
    "production": [
      "Asylum",
      "The"
    ],
    "poster_link": "https://images-na.ssl-images-amazon.com/images/M/MV5BMTQ2NjIxNTg5N15BMl5BanBnXkFtZTcwNzE0NzAzOA@@._V1_UY268_CR1,0,182,268_AL_.jpg"
  },
  {
    "rank": 1,
    "title": "Monster House (2006)",
    "summary": "Monsters under the bed are scary enough, but what happens when an entire house is out to get you? Three teens aim to find out when they go up against a decrepit neighboring home and unlock its frightening secrets.",
    "date": "07/21/2006",
    "genres": [
      "Animation",
      "Adventu